In [ ]:
import glob
import pandas as pd
import re
import subprocess

In [ ]:
# run with higgs combine tool

In [ ]:
outdir = 'assets_nov_2022'

In [ ]:
def make_file_df(pattern='txt'):
    '''This function produces a dataframe of all the cfg files that we could run combine over.'''
    file_list = [f for f in glob.glob('{}/combine_data/*/*BFFZprimeToMuMu*'.format(outdir)) if len(re.findall(pattern,f)) > 0]
    cfg_list =[]
    for i, f in enumerate(file_list):
        nJets = re.findall(r'SR.', f)[0]
        # don't include combined regions
        if not((nJets == "SR1") or (nJets == "SR2")): continue
        mass = (re.findall(r'M_(\d+)_', f)[0])
        sig_type = (re.findall(r'Mu_([a-z]+)_M_', f)[0])
        try:
            dbs = re.findall(r'dbs(\d)p(\d+)', f)[0]
        except:
            dbs = [0,0]
        dbs = ('{}.{}'.format(*dbs))
        era = (re.findall(r'(201.)',f)[0])
        if era=="201X": continue
        cfg_list.append({"file":f, "nJets":nJets, "mass":mass, "dbs":dbs, "era": era, "sig_type": sig_type})
    return pd.DataFrame(cfg_list)

In [ ]:
def make_combine_df(_df, _name):
    '''This function takes a dataframe of configs and produces a single datacard with those cards.'''
    files = _df.file
    file_string = "combineCards.py"
    for count, f in enumerate(files):
        file_string += " name{}={}".format(count, f)
    p = subprocess.Popen(file_string.split(' '), stdout=subprocess.PIPE, 
                                   stderr=subprocess.PIPE)
    out, err = p.communicate()
    era = "...." #re.findall(r'201[0-9]', out)[0]
    out = out.replace('{}/combine_data/2016/'.format(outdir), '')
    out = out.replace('{}/combine_data/2017/'.format(outdir), '')
    out = out.replace('{}/combine_data/2018/'.format(outdir), '')
    with open(_name, 'wb') as f:
        f.write(out)


In [ ]:
df = make_file_df()

In [ ]:
combined_regions = df[['mass', 'dbs', 'era']].drop_duplicates()

for i, (mass, dbs, era) in combined_regions.iterrows():
    tdf = df[(df.mass ==mass) & (df.dbs==dbs) & (df.era==era)]
    fname = "{}/combine_data/{}/{}_SRX_BFFZprimeToMuMu_fit_M_{}_dbs{}.txt".format(outdir, era, era, mass, dbs.replace('.', 'p'))
    make_combine_df(tdf, fname)

In [ ]:
combined_eras = df[['mass', 'dbs', 'nJets']].drop_duplicates()
for i, (mass, dbs, nJets) in combined_eras.iterrows():
    tdf = df[(df.mass ==mass) & (df.dbs==dbs) & (df.nJets==nJets)]
    fname = "{}/combine_data/201X/201X_{}_BFFZprimeToMuMu_fit_M_{}_dbs{}.txt".format(outdir, nJets, mass, dbs.replace('.', 'p'))
    make_combine_df(tdf, fname)

In [ ]:
combined_region_and_eras = df[['mass', 'dbs']].drop_duplicates()

for i, (mass, dbs) in combined_region_and_eras.iterrows():
    tdf = df[(df.mass ==mass) & (df.dbs==dbs)]
    fname = "{}/combine_data/201X/201X_SRX_BFFZprimeToMuMu_fit_M_{}_dbs{}.txt".format(outdir, mass, dbs.replace('.', 'p'))
    make_combine_df(tdf, fname)

In [ ]:
run_combine = '''#!/bin/sh
#ulimit -s unlimited
#set -e
cd /afs/cern.ch/work/r/rymuelle/public/nanoAODzPrime/higgscombine/CMSSW_10_2_13/src
export SCRAM_ARCH=slc7_amd64_gcc700
source /cvmfs/cms.cern.ch/cmsset_default.sh
eval `scramv1 runtime -sh`
cd /afs/cern.ch/work/r/rymuelle/public/nanoAODzPrime/CMSSW_12_1_0/src/bff_plotting_code_v3/{}/combine_data/ 

combine -M AsymptoticLimits "$1" --run blind 

'''.format(outdir)

In [ ]:
with open('{}/combine_data/run_combine.sh'.format(outdir), 'w') as f:
    f.write(run_combine)

In [ ]:
!cp src/combine_scripts/* {outdir}/combine_data/

In [ ]:
!mkdir {outdir}/combine_data/out
!mkdir {outdir}/combine_data/err
!mkdir {outdir}/combine_data/log

!mkdir {outdir}/combine_data/out/2016
!mkdir {outdir}/combine_data/err/2016
!mkdir {outdir}/combine_data/log/2016

!mkdir {outdir}/combine_data/out/2017
!mkdir {outdir}/combine_data/err/2017
!mkdir {outdir}/combine_data/log/2017


!mkdir {outdir}/combine_data/out/2018
!mkdir {outdir}/combine_data/err/2018
!mkdir {outdir}/combine_data/log/2018


!mkdir {outdir}/combine_data/out/201X
!mkdir {outdir}/combine_data/err/201X
!mkdir {outdir}/combine_data/log/201X

In [ ]:
condor_submit submit_jobs_201X.sub
condor_submit submit_jobs_2016.sub
condor_submit submit_jobs_2017.sub
condor_submit submit_jobs_2018.sub